# Binder Stress Test v2
This notebook runs multiple heavy computations to fully stress the Binder -> Docker -> Kubernetes environment.

In [ ]:
import numpy as np
from numba import njit, prange
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import time, multiprocessing

print("CPUs available:", multiprocessing.cpu_count())


In [ ]:
# 1. Bigger matrix multiplication
@njit(parallel=True)
def big_dot(A, B):
    n = A.shape[0]
    C = np.zeros((n, n))
    for i in prange(n):
        for j in range(n):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]
    return C

n = 1200
A = np.random.rand(n, n)
B = np.random.rand(n, n)

start = time.time()
C = big_dot(A, B)
elapsed = time.time() - start
print(f"Bigger matrix {n}x{n} multiplication elapsed: {elapsed:.2f} s")

plt.figure(figsize=(6,5))
plt.imshow(C[:100, :100], cmap='viridis')
plt.title(f"{n}x{n} Matrix Product Snapshot")
plt.colorbar()
plt.show()


In [ ]:
# 2. Multiple matrix multiplications in parallel
def mat_task(seed):
    np.random.seed(seed)
    A = np.random.rand(800, 800)
    B = np.random.rand(800, 800)
    return big_dot(A, B)

start = time.time()
results = Parallel(n_jobs=-1)(delayed(mat_task)(i) for i in range(4))
elapsed = time.time() - start
print(f"Parallel 4x 800x800 matrices elapsed: {elapsed:.2f} s")

plt.figure(figsize=(10,2))
for i, R in enumerate(results):
    plt.subplot(1,4,i+1)
    plt.imshow(R[:50,:50], cmap='plasma')
    plt.title(f"Task {i+1}")
plt.suptitle("Snapshots of parallel matrix products")
plt.show()


In [ ]:
# 3. Monte Carlo π estimation
@njit(parallel=True)
def monte_carlo_pi(num_samples):
    count = 0
    for i in prange(num_samples):
        x, y = np.random.rand(), np.random.rand()
        if x**2 + y**2 <= 1:
            count += 1
    return count

samples = 300_000_000  # 300 million samples
start = time.time()
hits = monte_carlo_pi(samples)
pi_est = 4 * hits / samples
elapsed = time.time() - start
print(f"Monte Carlo π estimation: {pi_est:.6f}, elapsed: {elapsed:.2f} s")


In [ ]:
# 4. SVD computation
from scipy.linalg import svd

X = np.random.rand(1000, 1000)
start = time.time()
U, s, V = svd(X)
elapsed = time.time() - start
print(f"SVD of 1000x1000 matrix elapsed: {elapsed:.2f} s")

plt.figure(figsize=(6,4))
plt.plot(s[:50], marker='o')
plt.title("Top 50 singular values of 1000x1000 matrix")
plt.xlabel("Index")
plt.ylabel("Singular value")
plt.grid(True)
plt.show()
